# Волны московской реновации 2.0

In [1]:
import pandas as pd
import numpy as np
import json
from tabula import read_pdf
from tqdm.notebook import tqdm
import os

Адреса и локации домов по реновации

In [2]:
with open('renovation_address.txt') as f:
    bounded_addresses = json.load(f)

In [3]:
def parse_pdf_table(pages, pdf_file='prikaz_grafikpereseleniya.pdf'):
    df = read_pdf(pdf_file, pages=pages, pandas_options={'header':None})
    
    # удаляем не нужные строки
    df = df[~(df.iloc[:,0] == 'No п/п')]
    
    df['unom'] = df.iloc[:,-1].combine_first(df.iloc[:,-2])
    
    # оставляем только нужные колонки
    df = df.iloc[:,[1, 2, 3, -1]]
    df.columns = ['AO', 'district', 'description', 'unom']
    
    return df

In [4]:
wave_1 = parse_pdf_table('3-29') # 2020 - 2024
wave_1['wave'] = 1

wave_2 = parse_pdf_table('30-76') # 2025 - 2028
wave_2['wave'] = 2

wave_3 = parse_pdf_table('77-128') # 2029 - 2032
wave_3['wave'] = 3

unknown = parse_pdf_table('129-148')
unknown['wave'] = 0

In [5]:
df = pd.concat([wave_1, wave_2, wave_3, unknown], ignore_index=True)

In [6]:
df['marker-color'] = df['wave'].map({1:'#0ACF00',  # зеленый
                                     2:'#1142AA',  # синий
                                     3:'#FFFD00',  # жёлтый
                                     0:'#FD0006'}) # красный

df['iconContent'] = df['wave'].map({1:'1',
                                    2:'2',
                                    3:'3',
                                    0:''})

In [7]:
df['longitude'] = 0
df['latitude'] = 0

In [8]:
for i in tqdm(bounded_addresses):
    unom = i['unom']
    coordinates = i['center']['coordinates']
    
    df.loc[df['unom']==unom, 'longitude'] = coordinates[1]
    df.loc[df['unom']==unom, 'latitude'] = coordinates[0]

In [9]:
# Объеденим ТАО и НАО в ТиНАО, т.к. в стартовых площадках есть только ТиНАО
df.loc[(df['AO'] == 'ТАО') | (df['AO'] == 'НАО'), 'AO'] = 'ТиНАО'

In [10]:
df[df['longitude'] == 0]

,AO,district,description,unom,wave,marker-color,iconContent,longitude,latitude
917,ТиНАО,поселение Михайлово-Ярцевское,"Армейский пос. (п.Михайлово-Ярцевское), д.11",15000016,1,#0ACF00,1,0.0,0.0
918,ТиНАО,поселение Михайлово-Ярцевское,"Армейский пос. (п.Михайлово-Ярцевское), д.13",15000015,1,#0ACF00,1,0.0,0.0
919,ТиНАО,поселение Михайлово-Ярцевское,"Армейский пос. (п.Михайлово-Ярцевское), д.3",15000013,1,#0ACF00,1,0.0,0.0
925,ТиНАО,поселение Михайлово-Ярцевское,"Армейский пос. (п.Михайлово-Ярцевское), д.4",15000012,1,#0ACF00,1,0.0,0.0
926,ТиНАО,поселение Михайлово-Ярцевское,"Армейский пос. (п.Михайлово-Ярцевское), д.6",15000014,1,#0ACF00,1,0.0,0.0
4883,ТиНАО,поселение Внуковское,"Гаражный пер. (пос.ДСК Мичуринец,п.Внуковское)...",4405823,0,#FD0006,,0.0,0.0
4945,ТиНАО,поселение Мосрентген,"Теплый Стан ул. (п.Мосрентген, в/г), д.51",20000002,0,#FD0006,,0.0,0.0
4946,ТиНАО,поселение Мосрентген,"Теплый Стан ул. (п.Мосрентген, в/г), д.52",20000003,0,#FD0006,,0.0,0.0
4947,ТиНАО,поселение Мосрентген,"Теплый Стан ул. (п.Мосрентген, в/г), д.53",20000001,0,#FD0006,,0.0,0.0
4948,ТиНАО,поселение Мосрентген,"Теплый Стан ул. (п.Мосрентген, в/г), д.85",20000000,0,#FD0006,,0.0,0.0


Добавляем вручную дома, которые не удалось спарсить

In [11]:
df.loc[917, ['longitude', 'latitude']] = 37.204805, 55.385382 
df.loc[918, ['longitude', 'latitude']] = 37.205255, 55.385367 
df.loc[919, ['longitude', 'latitude']] = 37.201518, 55.385265 
df.loc[925, ['longitude', 'latitude']] = 37.201545, 55.384927 
df.loc[926, ['longitude', 'latitude']] = 37.204151, 55.384576
df.loc[4883, ['longitude', 'latitude']] = 37.321218, 55.661308 
df.loc[4945, ['longitude', 'latitude']] = 37.476896, 55.604153 
df.loc[4946, ['longitude', 'latitude']] = 37.477406, 55.603895 
df.loc[4947, ['longitude', 'latitude']] = 37.476546, 55.602729 
df.loc[4948, ['longitude', 'latitude']] = 37.477568, 55.604659
df.loc[4995, ['longitude', 'latitude']] = 37.176806, 55.341541

Стартовые площадки

In [12]:
with open('start_area.txt') as f:
    end = json.load(f)

In [13]:
data = {
    'AO':[],
    'district':[],
    'longitude':[],
    'latitude':[],
    'description':[]
}

for i in end['response']:
    
    data['AO'].append(i['OKRUG'])
    
    data['district'] = i['AREA']
    
    coordinates = i['geoData']['coordinates']
    
    data['longitude'].append(coordinates[1])
    data['latitude'].append(coordinates[0])
    
    description = i['Address']
    
    if 'StartOfRelocation' in i:
        if i['StartOfRelocation'] is not None:
            description += '\n' + i['StartOfRelocation']
        
    data['description'].append(description)
    
df_start_area = pd.DataFrame(data)
df_start_area['marker-color'] = '#7D3E00' # коричневый цвет
df_start_area['iconContent'] = '0'
df_start_area['unom'] = None
df_start_area['wave'] = -1

Объеденяем метки домов по реновации и стартовых площадок

In [14]:
df = pd.concat([df, df_start_area], ignore_index=True)

Формирование карты реновации

In [15]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [16]:
properties = ['marker-color', 'iconContent', 'description']

Разделяем данные по округам.

In [17]:
if not os.path.exists('data'):
    os.makedirs('data')

for ao, data in df.groupby('AO'):
    geojson = df_to_geojson(data, properties)
    
    with open('data/' + ao + '.geojson', 'w') as f:
        json.dump(geojson, f, indent=2) 

Полная карта (может работать медленно)

In [18]:
geojson = df_to_geojson(df, properties)

with open('data/ВСЕ_ОКРУГА.geojson', 'w') as f:
    json.dump(geojson, f, indent=2) 